In [2]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import random as rd
import itertools
import time as t

def Game(Strat1,Strat2,LRounds, verbose=False) -> tuple[float,float,int,int]:
    benefit = 2
    cost = 1
    P1State=[None]*LRounds
    P2State=[None]*LRounds
    vP1State=[None]*LRounds
    Payoff1=0
    Payoff2=0

    TrueRecordP1, TrueRecordP2 = [], []

    for i in range(LRounds):
        Strat1(i,P1State,P2State)
        
        if i>0:
            vP1State[0:i]=P1State[0:i]           
        
        Strat2(i,P2State,vP1State)
        
        if P1State[i]=="c":
            Payoff1=Payoff1-cost
            Payoff2=Payoff2+benefit
            
        if P2State[i]=="c":
            Payoff2=Payoff2-cost
            Payoff1=Payoff1+benefit
            
        TrueRecordP1.append(P1State[i])
        TrueRecordP2.append(P2State[i])

    if Payoff1>Payoff2:
        Win1=1
        Win2=0
    elif Payoff2>Payoff1:
        Win1=0
        Win2=1
    else:
        Win1=0
        Win2=0
    
    if verbose:
        print('P1State : ', TrueRecordP1)
        print('P2State : ', TrueRecordP2)

    return Payoff1/LRounds,Payoff2/LRounds,Win1,Win2

def QuickGamePlays(Strat1,Strat2,LRounds) -> tuple[float,float,int,int]:
    P1State, P2State = [None]*LRounds, [None]*LRounds
    for i in range(LRounds):
        P1Statecopy, P2Statecopy = P1State.copy(), P2State.copy()
        P1State[i] = Strat1(i,P1Statecopy,P2Statecopy)[i]
        P2State[i] = Strat2(i,P2Statecopy,P1Statecopy)[i]
    return P1State, P2State

In [47]:
def cTFT(round,P1State,P2State):
    if round==0:
        P1State[round]="c"
    else:
        P1State[round]=P2State[round-1]
    return P1State
def dTFT(round,P1State,P2State):
    if round==0:
        P1State[round]="d"
    else:
        P1State[round]=P2State[round-1]
    return P1State
def TF2T(round,P1State,P2State):
    if round<2:
        P1State[round]="c"
    elif P2State[round-1]=="d" and P2State[round-2]=="d":
        P1State[round]="d"
    else:
        P1State[round]="c"
    return P1State
def T2FT(round,P1State,P2State):
    if round<2:
        P1State[round]="c"
    elif P2State[round-1]=="d" or P2State[round-2]=="d":
        P1State[round]="d"
    else:
        P1State[round]="c"
    return P1State
def GRIM(round,P1State,P2State):
    if "d" in P2State or "d" in P1State:
        P1State[round]="d"
    else:
        P1State[round]="c"
    return P1State
def GRADUAL(round,P1State,P2State):
    indicesD = [i for i, x in enumerate(P2State) if x == "d"]
    nDef2=len(indicesD)
    if round==0:
        P1State[round]="c"
    else:
        nDef1=0
        for j in range(nDef2):
            if P1State[round-j-1]=="d":
                nDef1=nDef1+1  
        if nDef2>nDef1:
            P1State[round]="d"
        else:
            P1State[round]="c"
    return P1State
def softMAJ(round,P1State,P2State):
    indicesD = [i for i, x in enumerate(P2State) if x == "d"]
    nDef2=len(indicesD)
    if round==0:
        P1State[round]="c"
    elif 2*nDef2<=round:
        P1State[round]="c"
    else:
        P1State[round]="d"        
    return P1State
def hardMAJ(round,P1State,P2State):
    indicesD = [i for i, x in enumerate(P2State) if x == "d"]
    nDef2=len(indicesD)
    if round==0:
        P1State[round]="d"
    elif 2*nDef2<round:
        P1State[round]="c"
    else:
        P1State[round]="d"       
    return P1State
def PROBER(round,P1State,P2State):
    if round==0:
        P1State[round]="d"
    elif round==1:
        P1State[round]="c"
    elif round==2:
        P1State[round]="c" 
    elif P2State[1]=="c" and P2State[2]=="c":
        P1State[round]="d"
    else:
        P1State[round]=P2State[round-1]
    return P1State
def GTFT(round,P1State,P2State):
    cost = 1
    benefit = 2
    if round==0:
        P1State[round]="c"
    elif P1State[round-1]=="c" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="d" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="c" and P2State[round-1]=="d":
        if random.rand()<1-cost/benefit:
            P1State[round]="c"
        else:
            P1State[round]="d"
    elif P1State[round-1]=="d" and P2State[round-1]=="d":
        if random.rand()<1-cost/benefit:
            P1State[round]="c"
        else:
            P1State[round]="d"
    return P1State
def dWSLS(round,P1State,P2State):
    if round==0:
        P1State[round]="d"
    elif P1State[round-1]=="c" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="d" and P2State[round-1]=="c":
        P1State[round]="d"
    elif P1State[round-1]=="c" and P2State[round-1]=="d":
        P1State[round]="d"
    elif P1State[round-1]=="d" and P2State[round-1]=="d":
        P1State[round]="c"
    return P1State
def cWSLS(round,P1State,P2State):
    if round==0:
        P1State[round]="c"
    elif P1State[round-1]=="c" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="d" and P2State[round-1]=="c":
        P1State[round]="d"
    elif P1State[round-1]=="c" and P2State[round-1]=="d":
        P1State[round]="d"
    elif P1State[round-1]=="d" and P2State[round-1]=="d":
        P1State[round]="c"
    return P1State
def ALLD(round,P1State,P2State):
    P1State[round]="d"
    return P1State
def ALLC(round,P1State,P2State):
    P1State[round]="c"
    return P1State
def RANDOM(round,P1State,P2State):
    if random.rand()<0.5:
        P1State[round]="c"
    else:
        P1State[round]="d"
    return P1State

In [65]:
def RATIO(round,P1State,P2State):
    if round == 0:
        P1State[round]="c"
    else:
        nd = P2State.count("d") # no. of defects
        nc = P2State.count("c") # no. of cooperations
        if nd == 0:
            P1State[round]="c"
        else:
            r = nc/nd
            if r > 0.925:
                P1State[round]="c"
            else:
                P1State[round]="d"
    return P1State

def ScoreTeam10(SuperSecretCodes='random'):
    # SuperSecretCodes[0] = code for ScoreRunner, SuperSecretCodes[1] = code for ScorePaceMaker
    if SuperSecretCodes == 'random':
        SuperSecretCodes = [list(rd.choice(list(itertools.product(['c','d'],repeat=10)))), list(rd.choice(list(itertools.product(['c','d'],repeat=10))))]
        print("SuperSecretCodes: ",SuperSecretCodes)
    
    RunnerMoves, PaceMakerMoves = SuperSecretCodes[0] + ['d']*(200 - len(SuperSecretCodes[0])), SuperSecretCodes[1] + ['c']*(200 - len(SuperSecretCodes[1]))
    # In case runner goes up against runner
    # RRMoves = SuperSecretCodes[0] + ['d']*(200 - len(SuperSecretCodes[0]))

    def kindness(round):
        return 1 - (0.8)*(199-round)

    def ScoreRunner10(round,P1State,P2State):
        # Is the opponent a ScorePaceMaker?
        if P2State[:round] == PaceMakerMoves[:round]:
            P1State[round] = RunnerMoves[round]
            return P1State
        
        # # Is the opponent a ScoreRunner?
        # if P2State[:round] == RRMoves[:round]:
        #     P1State[round] = RRMoves[round]
        #     return P1State
        
        # Otherwise, play RATIO -- Placeholder strategy for now
        P1State[round] = RATIO(round,P1State,P2State)[round]
        
        # Do I want to be kind?
        if P1State[round] == "c" and rd.random() < kindness(round):
            P1State[round] = "d"

        return P1State

    def ScorePaceMaker10(round,P1State,P2State):
        # Is the opponent a ScoreRunner?
        if P2State[:round] == RunnerMoves[:round]:
            P1State[round] = PaceMakerMoves[round]
            return P1State
        
        # Otherwise, play ALLD
        P1State[round]="d"
        return P1State
    
    return ScoreRunner10,ScorePaceMaker10

def ScoreTeam8plus(SuperSecretCodes='random'):
    lenCode = 8
    # SuperSecretCodes[0] = code for ScoreRunner, SuperSecretCodes[1] = code for ScorePaceMaker
    if SuperSecretCodes == 'random':
        SuperSecretCodes = [list(rd.choice(list(itertools.product(['c','d'],repeat=lenCode)))), list(rd.choice(list(itertools.product(['c','d'],repeat=lenCode))))]
        print("SuperSecretCodes: ",SuperSecretCodes)
    
    PaceMakerMoves = SuperSecretCodes[1] + ['c']*(200 - lenCode)
    RunnerMoves = QuickGamePlays(RATIO, (lambda a, b, c : PaceMakerMoves), lenCode)[0] + SuperSecretCodes[0] + ['d']*(200 - 2*lenCode)

    def ScoreRunner8plus(nthround,P1State,P2State):
        # Is the opponent a ScorePaceMaker?
        if P2State[:max(lenCode, nthround)] == PaceMakerMoves[:max(lenCode, nthround)]:
            P1State[nthround] = RunnerMoves[nthround]
            return P1State
        
        # Otherwise, play RATIO
        P1State[nthround] = RATIO(nthround,P1State,P2State)[nthround]
        
        # Do I want to be kind?
        if P1State[nthround] == "c" and rd.random() < 1 - (0.8)*(199-nthround):
            P1State[nthround] = "d"

        return P1State

    def ScorePaceMaker8plus(nthround,P1State,P2State):
        # Is the opponent a ScoreRunner?
        if P2State[:nthround] == RunnerMoves[:nthround]:
            P1State[nthround] = PaceMakerMoves[nthround]
            return P1State
        
        # Otherwise, play ALLD
        P1State[nthround]="d"
        return P1State
    
    return ScoreRunner8plus, ScorePaceMaker8plus

def ScoreTeamn(lenCode = 8, SuperSecretCodes='random'):
    # SuperSecretCodes[0] = code for ScoreRunner, SuperSecretCodes[1] = code for ScorePaceMaker
    if SuperSecretCodes == 'random':
        SuperSecretCodes = [list(rd.choice(list(itertools.product(['c','d'],repeat=lenCode)))), list(rd.choice(list(itertools.product(['c','d'],repeat=lenCode))))]
        print("SuperSecretCodes: ",SuperSecretCodes)
    
    PaceMakerMoves = SuperSecretCodes[1] + ['c']*(200 - lenCode)
    RunnerMoves = QuickGamePlays(RATIO, (lambda a, b, c : PaceMakerMoves), lenCode)[0] + SuperSecretCodes[0] + ['d']*(200 - 2*lenCode)

    def ScoreRunner8plus(nthround,P1State,P2State):
        # Is the opponent a ScorePaceMaker?
        if P2State[:max(lenCode, nthround)] == PaceMakerMoves[:max(lenCode, nthround)]:
            P1State[nthround] = RunnerMoves[nthround]
            return P1State
        
        # Otherwise, play RATIO
        P1State[nthround] = RATIO(nthround,P1State,P2State)[nthround]
        
        # Do I want to be kind?
        if P1State[nthround] == "c" and rd.random() < 1 - (0.8)*(199-nthround):
            P1State[nthround] = "d"

        return P1State

    def ScorePaceMaker8plus(nthround,P1State,P2State):
        # Is the opponent a ScoreRunner?
        if P2State[:nthround] == RunnerMoves[:nthround]:
            P1State[nthround] = PaceMakerMoves[nthround]
            return P1State
        
        # Otherwise, play ALLD
        P1State[nthround]="d"
        return P1State
    
    return ScoreRunner8plus, ScorePaceMaker8plus

def ScoreTeam9(SuperSecretCodes='random'):
    # SuperSecretCodes[0] = code for ScoreRunner, SuperSecretCodes[1] = code for ScorePaceMaker
    if SuperSecretCodes == 'random':
        SuperSecretCodes = [list(rd.choice(list(itertools.product(['c','d'],repeat=9)))), list(rd.choice(list(itertools.product(['c','d'],repeat=9))))]
        print("SuperSecretCodes: ",SuperSecretCodes)
    
    RunnerMoves, PaceMakerMoves = SuperSecretCodes[0] + ['d']*(200 - len(SuperSecretCodes[0])), SuperSecretCodes[1] + ['c']*(200 - len(SuperSecretCodes[1]))
    # In case runner goes up against runner
    # RRMoves = SuperSecretCodes[0] + ['d']*(200 - len(SuperSecretCodes[0]))

    def kindness(round):
        return 1 - (0.8)*(199-round)

    def ScoreRunner9(round,P1State,P2State):
        # Is the opponent a ScorePaceMaker?
        if P2State[:round] == PaceMakerMoves[:round]:
            P1State[round] = RunnerMoves[round]
            return P1State
        
        # # Is the opponent a ScoreRunner?
        # if P2State[:round] == RRMoves[:round]:
        #     P1State[round] = RRMoves[round]
        #     return P1State
        
        # Otherwise, play RATIO -- Placeholder strategy for now
        P1State[round] = RATIO(round,P1State,P2State)[round]
        
        # Do I want to be kind?
        if P1State[round] == "c" and rd.random() < kindness(round):
            P1State[round] = "d"

        return P1State

    def ScorePaceMaker9(round,P1State,P2State):
        # Is the opponent a ScoreRunner?
        if P2State[:round] == RunnerMoves[:round]:
            P1State[round] = PaceMakerMoves[round]
            return P1State
        
        # Otherwise, play ALLD
        P1State[round]="d"
        return P1State
    
    return ScoreRunner9, ScorePaceMaker9

def ScoreTeam8(SuperSecretCodes='random'):
    LenCode = 8
    # SuperSecretCodes[0] = code for ScoreRunner, SuperSecretCodes[1] = code for ScorePaceMaker
    if SuperSecretCodes == 'random':
        SuperSecretCodes = [list(rd.choice(list(itertools.product(['c','d'],repeat=LenCode)))), list(rd.choice(list(itertools.product(['c','d'],repeat=LenCode))))]
        print("SuperSecretCodes: ",SuperSecretCodes)
    
    RunnerMoves, PaceMakerMoves = SuperSecretCodes[0] + ['d']*(200 - len(SuperSecretCodes[0])), SuperSecretCodes[1] + ['c']*(200 - len(SuperSecretCodes[1]))
    # In case runner goes up against runner
    # RRMoves = SuperSecretCodes[0] + ['d']*(200 - len(SuperSecretCodes[0]))

    def kindness(round):
        return 1 - (0.8)*(199-round)

    def ScoreRunner8(round,P1State,P2State):
        # Is the opponent a ScorePaceMaker?
        if P2State[:round] == PaceMakerMoves[:round]:
            P1State[round] = RunnerMoves[round]
            return P1State
        
        # # Is the opponent a ScoreRunner?
        # if P2State[:round] == RRMoves[:round]:
        #     P1State[round] = RRMoves[round]
        #     return P1State
        
        # Otherwise, play RATIO -- Placeholder strategy for now
        P1State[round] = RATIO(round,P1State,P2State)[round]
        
        # Do I want to be kind?
        if P1State[round] == "c" and rd.random() < kindness(round):
            P1State[round] = "d"

        return P1State

    def ScorePaceMaker8(round,P1State,P2State):
        # Is the opponent a ScoreRunner?
        if P2State[:round] == RunnerMoves[:round]:
            P1State[round] = PaceMakerMoves[round]
            return P1State
        
        # Otherwise, play ALLD
        P1State[round]="d"
        return P1State
    
    return ScoreRunner8, ScorePaceMaker8

def ScoreTeam7(SuperSecretCodes='random'):
    # SuperSecretCodes[0] = code for ScoreRunner, SuperSecretCodes[1] = code for ScorePaceMaker
    if SuperSecretCodes == 'random':
        SuperSecretCodes = [list(rd.choice(list(itertools.product(['c','d'],repeat=7)))), list(rd.choice(list(itertools.product(['c','d'],repeat=7))))]
        print("SuperSecretCodes: ",SuperSecretCodes)
    
    RunnerMoves, PaceMakerMoves = SuperSecretCodes[0] + ['d']*(200 - len(SuperSecretCodes[0])), SuperSecretCodes[1] + ['c']*(200 - len(SuperSecretCodes[1]))
    # In case runner goes up against runner
    # RRMoves = SuperSecretCodes[0] + ['d']*(200 - len(SuperSecretCodes[0]))

    def kindness(round):
        return 1 - (0.8)*(199-round)

    def ScoreRunner7(round,P1State,P2State):
        # Is the opponent a ScorePaceMaker?
        if P2State[:round] == PaceMakerMoves[:round]:
            P1State[round] = RunnerMoves[round]
            return P1State
        
        # # Is the opponent a ScoreRunner?
        # if P2State[:round] == RRMoves[:round]:
        #     P1State[round] = RRMoves[round]
        #     return P1State
        
        # Otherwise, play RATIO -- Placeholder strategy for now
        P1State[round] = RATIO(round,P1State,P2State)[round]
        
        # Do I want to be kind?
        if P1State[round] == "c" and rd.random() < kindness(round):
            P1State[round] = "d"

        return P1State

    def ScorePaceMaker7(round,P1State,P2State):
        # Is the opponent a ScoreRunner?
        if P2State[:round] == RunnerMoves[:round]:
            P1State[round] = PaceMakerMoves[round]
            return P1State
        
        # Otherwise, play ALLD
        P1State[round]="d"
        return P1State
    
    return ScoreRunner7, ScorePaceMaker7

def ScoreTeam6(SuperSecretCodes='random'):
    lenCode = 6
    # SuperSecretCodes[0] = code for ScoreRunner, SuperSecretCodes[1] = code for ScorePaceMaker
    if SuperSecretCodes == 'random':
        SuperSecretCodes = [list(rd.choice(list(itertools.product(['c','d'],repeat=lenCode)))), list(rd.choice(list(itertools.product(['c','d'],repeat=lenCode))))]
        print("SuperSecretCodes: ",SuperSecretCodes)
    
    RunnerMoves, PaceMakerMoves = SuperSecretCodes[0] + ['d']*(200 - len(SuperSecretCodes[0])), SuperSecretCodes[1] + ['c']*(200 - len(SuperSecretCodes[1]))
    # In case runner goes up against runner
    # RRMoves = SuperSecretCodes[0] + ['d']*(200 - len(SuperSecretCodes[0]))

    def kindness(round):
        return 1 - (0.8)*(199-round)

    def ScoreRunner6(round,P1State,P2State):
        # Is the opponent a ScorePaceMaker?
        if P2State[:round] == PaceMakerMoves[:round]:
            P1State[round] = RunnerMoves[round]
            return P1State
        
        # # Is the opponent a ScoreRunner?
        # if P2State[:round] == RRMoves[:round]:
        #     P1State[round] = RRMoves[round]
        #     return P1State
        
        # Otherwise, play RATIO -- Placeholder strategy for now
        P1State[round] = RATIO(round,P1State,P2State)[round]
        
        # Do I want to be kind?
        if P1State[round] == "c" and rd.random() < kindness(round):
            P1State[round] = "d"

        return P1State

    def ScorePaceMaker6(round,P1State,P2State):
        # Is the opponent a ScoreRunner?
        if P2State[:round] == RunnerMoves[:round]:
            P1State[round] = PaceMakerMoves[round]
            return P1State
        
        # Otherwise, play ALLD
        P1State[round]="d"
        return P1State
    
    return ScoreRunner6, ScorePaceMaker6

def TeamOrganise(ScoreRunner, ScorePaceMaker):
    return [ScoreRunner]*2 + [ScorePaceMaker]*7

ST10plus = TeamOrganise(*ScoreTeam8plus())
ST10 = TeamOrganise(*ScoreTeam10())
ST9 = TeamOrganise(*ScoreTeam9())
ST8 = TeamOrganise(*ScoreTeam8())
ST7 = TeamOrganise(*ScoreTeam7())
ST6 = TeamOrganise(*ScoreTeam6())


def extort(round,P1State,P2State):
    p_dict = {('c', 'c') : 8/9, ('c', 'd') : 1/2, ('d', 'c') : 1/3, ('d', 'd') : 0}
    if round == 0 : 
        P1State[round]="c"
    elif rd.random() < p_dict[(P1State[round-1], P2State[round-1])]:
        P1State[round]="c"
    else:
        P1State[round]="d"
    return P1State

def extortD(nthround,P1State,P2State):
    if nthround < 100:
        P1State[nthround] = extort(nthround,P1State,P2State)[nthround]
    else:
        P1State[nthround] = "d"
    return P1State


def guesser1(P1State,P2State):
    history = list(zip(P1State,P2State))
    c_history = [history[index-1] for index, i in enumerate(history) if i[1] == 'c']
    p_cc = c_history.count(('c','c')) / history.count(('c','c')) if history.count(('c','c')) != 0 else 1
    p_cd = c_history.count(('c','d')) / history.count(('c','d')) if history.count(('c','d')) != 0 else 1
    p_dc = c_history.count(('d','c')) / history.count(('d','c')) if history.count(('d','c')) != 0 else 1
    p_dd = c_history.count(('d','d')) / history.count(('d','d')) if history.count(('d','d')) != 0 else 1/2
    return p_cc,p_cd,p_dc,p_dd

def confidence1(P1State,P2State):
    history = list(zip(P1State,P2State))
    possible1history = [('c', 'c'), ('c', 'd'), ('d', 'c'), ('d', 'd')]
    confidence = 0
    for prev in possible1history:
        prev_history = [i[1] == 'c' for index, i in enumerate(history) if history[index-1] == prev]
        if len(prev_history) != 0:
            confidence += 2 * len([prev_history[index] != prev_history[index-1] for index, i in enumerate(prev_history) if index != 0]) / len(prev_history)
        else:
            confidence += 0.5
    return confidence/4

def simulate(p, x, startRound, endRound, initial_state=np.array([1/2, 1/2, 0/4, 0/4]).T):
    M = np.array([[p[i]*x[i], (p[i])*(1-x[i]), (1-p[i])*(x[i]), (1-p[i])*(1-x[i])] for i in range(4)]).T
    score = 0
    for _ in range(min(30, endRound-startRound)):
        initial_state = M @ initial_state
        score += np.array([1, -1, 2, 0]) @ initial_state
    return score

# BestResponse = np.loadtxt("BestResponse8.1.txt")
# print(BestResponse[1])

def Guesser1(round,P1State,P2State):
    if round < 30 : 
        P1State[round] = RATIO(round,P1State,P2State)[round]
        return P1State

    p_cc,p_cd,p_dc,p_dd = guesser1(P1State,P2State)
    p = np.array([p_cc, p_cd, p_dc, p_dd])

    previous_state = np.array([0, 0, 0, 0])
    previous_state[2*(P1State[round-1] == 'd') + (P2State[round-1] == 'd')] = 1
    
    n = 2
    X = np.array(np.meshgrid(np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n))).T.reshape(-1,4)
    scores = np.array([simulate(x, p, round+1, 200, initial_state=previous_state.T) for x in X[::-1]])
    best_x = X[::-1][np.argmax(scores)]
    # print(best_x, scores[np.argmax(scores)])
    # best_x = BestResponse[np.argmin(np.linalg.norm(BestResponse[:,:4] - p, axis=1))][4:]

    if rd.random() < best_x@previous_state:
        P1State[round] = 'c'
    else:
        P1State[round] = 'd'
    return P1State

def forgetfulRATIO(round,P1State,P2State):
    flip = P2State[:round][::-1]

    def forgetfulness(round):
        if round < 9:
            return 1
        else:
            return 0.5**(round-8) + 1/2
    
    nd = sum([forgetfulness(i) for i, x in enumerate(flip) if x == "d"])
    nc = sum([forgetfulness(i) for i, x in enumerate(flip) if x == "c"])
    if nd == 0:
        P1State[round]="c"
    elif nc/nd > 0.925:
        P1State[round]="c"
    else:
        P1State[round]="d"

    if P1State[round] == "c" and rd.random() > 1 - (0.5)**(199-round):
        P1State[round] = "d"
    return P1State

def RATIOShort(nthround,P1State,P2State):
    nd = P2State.count("d") # no. of defects
    nc = nthround - nd      # no. of cooperations
    if nd == 0:
        P1State[nthround]="c"
    elif nc/nd > 0.925:
        P1State[nthround]="c"
    else:
        P1State[nthround]="d"
    return P1State

def RATIOTFT(round,P1State,P2State):
    if round == 0:
        P1State[round] = "c"
    else:
        # count number of defect and cooperate
        nd = P2State.count("d")
        nc = P2State.count("c")
        
        if nd == 0:
            P1State[round] = "c"
        else:
            r = nc/nd
            if r > 0.925:
                P1State[round]="c" # cooperate regardless
            else:
                P1State[round]=P2State[round-1] # play TFT
    return P1State


def RATIOk5(round,P1State,P2State):
    if round == 0:
        P1State[round]="c"
    else:
        nd = P2State.count("d") # no. of defects
        nc = P2State.count("c") # no. of cooperations
        if nd == 0:
            P1State[round]="c"
        else:
            r = nc/nd
            if r > 0.925:
                P1State[round]="c"
            else:
                P1State[round]="d"
    
    if P1State[round] == "c" and rd.random() > 1 - (0.5)**(199-round):
        P1State[round] = "d"
    return P1State

def ResponseDecider(round,P1State,P2State):
    response = [0]
    period = 10
    for _ in range(round//period):
        reaction, P2State = P2State[:period], P2State[period:]
        nd = reaction.count("d") # no. of defects
        if nd <= 1:
            response.append(-1)
        elif nd <= 4:
            response.append(0)
        else:
            response.append(1)
    return response[1:]

def TieredResponse(nthround,P1State,P2State):
    strats = [ALLD, extort, RATIO, ALLC]
    stratIndex = 1
    decisions = ResponseDecider(nthround,P1State,P2State)
    for i in decisions:
        if stratIndex == 0 and i == 1:
            stratIndex = 1
            strats = strats[1:]
        else:
            stratIndex += i
            
        if stratIndex < 0:
            stratIndex = 0
        elif stratIndex == len(strats):
            P1State = ALLD(nthround,P1State,P2State)
            return P1State
        
        # if round == 199:
        #     print(stratIndex)

    if P2State[:10] == ['d'] * 10:
        P1State = ALLD(nthround,P1State,P2State)
        return P1State

    P1State = strats[round(stratIndex)](nthround,P1State,P2State)
    return P1State
    

def Oracle1(nthround, P1State, P2State):
    if nthround < 10:
        P1State = RATIO(nthround,P1State,P2State)
        return P1State
    
    p_cc,p_cd,p_dc,p_dd = guesser1(P1State[nthround//2+2:],P2State[nthround//2+2:])
    p = np.array([p_cc, p_cd, p_dc, p_dd])

    previous_state = np.array([0, 0, 0, 0])
    previous_state[2*(P1State[nthround-1] == 'd') + (P2State[nthround-1] == 'd')] = 1
    
    n = 2
    X = np.array(np.meshgrid(np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n))).T.reshape(-1,4)
    scores = np.array([simulate(x, p, nthround+1, 200, initial_state=previous_state.T) for x in X[::-1]])
    best_x = X[::-1][np.argmax(scores)]
    # print(best_x, scores[np.argmax(scores)])
    # best_x = BestResponse[np.argmin(np.linalg.norm(BestResponse[:,:4] - p, axis=1))][4:]

    if rd.random() < best_x@previous_state:
        P1State[nthround] = 'c'
    else:
        P1State[nthround] = 'd'

    return P1State

def ALT(round,P1State,P2State): #ALT = alternating cooperate and defect
    if round == 0:
        P1State[round]="c"
    elif P1State[round - 1] == "c":
        P1State[round]="d"
    else:
        P1State[round]="c"
    return P1State

def RATIO2(round,P1State,P2State): #RATIO = cooperates based on the ratio of cooperation to defection
    if round == 0:
        P1State[round]="c"
    elif round < 50:
        nd = P2State.count("d") # no. of defects
        nc = P2State.count("c") # no. of cooperations
        if nd == 0:
            P1State[round]="c"
        else:
            r = nc/nd
            if r > 0.925:
                P1State[round]="c";
            else:
                P1State[round]="d";
    elif round >= 50 and round < 100:
        P1State[round]="c";
    elif round >= 100:
        nd = P2State.count("d") # no. of defects
        nc = P2State.count("c") # no. of cooperations
        if nd == 0:
            P1State[round]="c"
        else:
            r = nc/nd
            if r > 0.925:
                P1State[round]="c";
            else:
                P1State[round]="d";
    else:
        P1State[round]="d";
    return P1State

# Jaewon's strategy
def cT20FT(round, P1State, P2State):
    if round == 0:
        P1State[round] = "c"  # Start by cooperating
    else:
        # Cooperate until opponent defects, then defect for a few rounds
        if "d" in P2State[max(0, round - 20):round]:
            P1State[round] = "d"
        else:
            P1State[round] = "c"
    return P1State

def PAD1(round,P1State,P2State): #PAD1 = Probability adjusted 1st iteration
    if round == 0:
        P1State[round]="c"
    else:
        p = 0.5
        nd = P2State.count("d") # no. of defects
        nc = P2State.count("c") # no. of cooperations
        p = p - nd*0.2
        p = p + nc*0.2
        if p > 1:
            p = 1
        elif p < 0:
            p = 0
        if random.rand()<p:
            P1State[round]="c";
        else:
            P1State[round]="d";
    return P1State

def PAD10(nthround,P1State,P2State):
    if nthround < 10:
        P1State = RATIO(nthround,P1State,P2State)
        return P1State
    
    response = []
    rereaction = [-0.1, 0, 0.1, 0.15, 0.2, 0.25, 0.3, 0.3, 0.3, -0.3, -100]
    period = 10
    for _ in range(nthround//period):
        reaction, P2State = P2State[:period], P2State[period:]
        nd = reaction.count("d") # no. of defects
        response.append(rereaction[nd])

    p = 1 + sum(response)
    if p > 1.5 and response[-1] >= 0.25:
        P1State[nthround] = "d"
    elif random.rand() < p:
        P1State[nthround]="c"
    else:
        P1State[nthround]="d"
    return P1State

def Update_Strat(round,P1State,P2State):          #Idea is to take the way we did updating mixed strategy profiles in lecutres 
    
    def payoff(p1, p2):
        return p1 - 2*p2 + 1
    
    if round == 0:
        P1State[round] = "c"     #d vs c seems to not matter too much
        return P1State
    
    #GET P2 ON AVERAGE
    
    n2c = P2State.count("c")
    n2d = P2State.count("d")
    p2_av = (0* n2c + 1 * n2d)/round
           
        
    #GET P1 ON AVERAGE
    n1c = P1State.count("c")
    n1d = P1State.count("d")
    p1_av = (0* n1c + 1 * n1d)/round
           
        
    #UPDATING P1
    phi_D = np.max([0, payoff(1, p2_av) - payoff(p1_av, p2_av)])
    phi_C = np.max([0, payoff(0, p2_av) - payoff(p1_av, p2_av)])
    
    p1 = (payoff(1, p2_av) + phi_D)/(1 + phi_D + phi_C)
    
    if random.rand()>p1:
        P1State[round]="c";
    else:
        P1State[round]="d";
        
    return P1State

def squared_prop(round,P1State,P2State):
    
    if round == 0:
        P1State[round] = "c"     #garner good will
        return P1State
    
    p2_type = [P2State[0]][0]    #determine which unique is being counted
    
    p2_av_arr = np.unique(P2State[:round], return_counts = "True")   #get array of no. of C vs no. of D
    
    
    if len(p2_av_arr[1]) == 1:     #if all entires are the same
        if P2State[0] == "d":
            p2_av = 1
        else:
            p2_av = 0
            
    else:
        p2_av = (0*p2_av_arr[1][0] + 1*p2_av_arr[1][1])/round      #probability that p2 has defected
        
        
    p1 = p2_av**2      #make p1 some function proportional to average of the p2’s s.t the function has a positive 2nd derivative to minimise payoff for u2 potentially
    
    if random.rand()>p1:
        P1State[round]="c";
    else:
        P1State[round]="d";
            
    return P1State

def exp_prop(round,P1State,P2State):
    
    if round == 0:
        P1State[round] = "c"     #garner good will
        return P1State
    
    p2_type = [P2State[0]][0]    #determine which unique is being counted
    
    p2_av_arr = np.unique(P2State[:round], return_counts = "True")   #get array of no. of C vs no. of D
    
    
    if len(p2_av_arr[1]) == 1:     #if all entires are the same
        if P2State[0] == "d":
            p2_av = 1
        else:
            p2_av = 0
            
    else:
        p2_av = (0*p2_av_arr[1][0] + 1*p2_av_arr[1][1])/round      #probability that p2 has defected
        
        
    p1 = (np.exp(p2_av)-1)/(np.exp(1)-1)
    
    if random.rand()>p1:
        P1State[round]="c";
    else:
        P1State[round]="d";

    if P1State[round] == "c" and rd.random() > 1 - (0.5)**(199-round):
        P1State[round] = "d"
    return P1State

def ForgivingSoftMAJ(nthround,P1State,P2State):
    if nthround == 27:
        nc2 = 2*P2State.count("c")
        if nc2 >= P1State.count("c") and nc2 >= nthround - 1:
            P1State[nthround] = "c"
            return P1State

    if nthround >= 197:
        P1State[nthround] = "d"
        return P1State

    P1State[nthround] = softMAJ(nthround,P1State,P2State)[nthround]
    return P1State

def ForgivingCTFT(nthround,P1State,P2State):
    if nthround == 27:
        nc2 = 2*P2State.count("c")
        if nc2 >= P1State.count("c") and nc2 >= nthround - 1:
            P1State[nthround] = "c"
            return P1State
        
    if nthround >= 197:
        P1State[nthround] = "d"
        return P1State

    P1State[nthround] = cTFT(nthround,P1State,P2State)[nthround]
    return P1State

def forgetfulsoftMAJ(nthround,P1State,P2State):
    flip = P2State[:nthround][::-1]

    def forgetfulness(round):
        if round < 9:
            return 1
        else:
            return 0.5**(round-8) + 1/2
    
    nd = sum([forgetfulness(i) for i, x in enumerate(flip) if x == "d"])
    nc = sum([forgetfulness(i) for i, x in enumerate(flip) if x == "c"])

    P1State[nthround] = "c" if 2*nd <= (nd + nc) else "d"
    return P1State

def ForgivingSoftMAJ(nthround,P1State,P2State):
    if nthround == 27:
        nc2 = 2*P2State.count("c")
        if nc2 >= P1State.count("c") and nc2 >= nthround - 1:
            P1State[nthround] = "c"
            return P1State

    P1State[nthround] = forgetfulsoftMAJ(nthround,P1State,P2State)[nthround]
    return P1State

def ScoreRunnerFinal(nthround,P1State,P2State):
    PaceMakerMoves = ['d', 'd', 'c', 'c', 'c'] + ['c']*(200 - 5)
    RunnerMoves = ['c', 'd', 'd', 'd', 'c', 'c', 'd', 'c'] + ['d']*(200 - 8)

    # Is the opponent a ScorePaceMaker?
    if P2State[:max(5, nthround)] == PaceMakerMoves[:max(5, nthround)]:
        P1State[nthround] = RunnerMoves[nthround]
        return P1State

    if 180 - nthround < 1:
        P1State[nthround] = "d"
        return P1State

    # Otherwise, play RATIO
    P1State[nthround] = ForgivingSoftMAJ(nthround,P1State,P2State)[nthround]
    return P1State

def ScorePaceMakerFinal(nthround,P1State,P2State):
    PaceMakerMoves = ['d', 'd', 'c', 'c', 'c'] + ['c']*(200 - 5)
    RunnerMoves = ['c', 'd', 'd', 'd', 'c', 'c', 'd', 'c'] + ['d']*(200 - 8)

    # Is the opponent a ScoreRunner?
    if P2State[:nthround] == RunnerMoves[:nthround]:
        P1State[nthround] = PaceMakerMoves[nthround]
        return P1State
    
    # Otherwise, play ALLD
    P1State[nthround]="d"
    return P1State

# def confidence1mem(P1State, P2State):
#     history = list(zip(P1State,P2State))
#     possible1history = [('c', 'c'), ('c', 'd'), ('d', 'c'), ('d', 'd')]
#     confidence = 0
#     for prev in possible1history:
#         prev_history = [i[1] == 'c' for index, i in enumerate(history) if history[index-1] == prev]
#         if len(prev_history) != 0:
#             confidence += 2 * len([prev_history[index] != prev_history[index-1] for index, i in enumerate(prev_history) if index != 0]) / len(prev_history)
#         else:
#             confidence += 0.5
#     return confidence/4

# def chainsquare(State):
#     return sum(sum(1 for _ in i[1])**2 for i in itertools.groupby(State))


# def f(p : float, numround : int) -> float:
#     res = 0
#     for i in itertools.product(['c','d'],repeat=numround):
#         nc = i.count('c')
#         res += p**nc * (1-p)**(numround-nc) * chainsquare(i)
    # return res

# for i in np.linspace(0.3, 0.5, 11):
#     print(round(f(i, 20) - f(i, 19), 4), end=', ')


# for p in np.linspace(0.1, 0.5, 5):
#     for i in range(1, 21):
#         print(round(f(p, i), 4), end=', ')
#     print()

# print(f(0.42, 22)- f(0.42, 21))

# def fquick(p : float, numround : int) -> float:
# [1.0, 3.0, 5.5, 8.25, 11.125, 14] +3

SuperSecretCodes:  [['c', 'd', 'd', 'd', 'd', 'd', 'c', 'c'], ['c', 'd', 'c', 'd', 'c', 'd', 'd', 'c']]
SuperSecretCodes:  [['d', 'd', 'c', 'c', 'd', 'c', 'c', 'c', 'd', 'c'], ['c', 'c', 'd', 'c', 'd', 'd', 'c', 'd', 'd', 'c']]
SuperSecretCodes:  [['c', 'c', 'c', 'd', 'c', 'd', 'c', 'd', 'd'], ['c', 'd', 'd', 'c', 'd', 'd', 'd', 'd', 'c']]
SuperSecretCodes:  [['d', 'c', 'd', 'd', 'd', 'd', 'd', 'd'], ['d', 'd', 'd', 'c', 'c', 'd', 'd', 'd']]
SuperSecretCodes:  [['d', 'd', 'c', 'd', 'c', 'c', 'd'], ['d', 'd', 'c', 'c', 'd', 'c', 'c']]
SuperSecretCodes:  [['d', 'c', 'd', 'c', 'c', 'd'], ['c', 'd', 'd', 'c', 'c', 'd']]


In [50]:
for stratOpponent in [RATIO, TieredResponse, cTFT,dTFT,cWSLS,dWSLS,GTFT,TF2T,T2FT,GRIM,GRADUAL,softMAJ,hardMAJ,PROBER, ALT, RATIO2, cT20FT, PAD1, Update_Strat, squared_prop, exp_prop, PAD10, forgetfulRATIO, RATIOk5, exp_prop_sqrt, extort, extortD, ALLD]:
    print(f'stratOpponent is {stratOpponent.__name__} :')
    print(Game(softMAJ, stratOpponent, 200, verbose=True))
    print(Game(ForgivingSoftMAJ, stratOpponent, 200, verbose=True))
    print(Game(RATIOk5, stratOpponent, 200, verbose=True))
    print(Game(ForgivingCTFT, stratOpponent, 200, verbose=True))

stratOpponent is RATIO :
P1State :  ['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c'

In [10]:
# StratList = [RATIO, TieredResponse, cTFT,dTFT,cWSLS,dWSLS,GTFT,TF2T,T2FT,GRIM,GRADUAL,softMAJ,hardMAJ,PROBER, RATIOTFT, ALT, cT20FT, Update_Strat, squared_prop, exp_prop]*3 + [extort, ALLD]*30
# StratList += TeamOrganise(*ScoreTeam8plus())
# StratList += TeamOrganise(*ScoreTeam10())
# StratList += TeamOrganise(*ScoreTeam9())
# StratList += TeamOrganise(*ScoreTeam8())
# StratList += TeamOrganise(*ScoreTeam7())
# StratList += TeamOrganise(*ScoreTeam6())
StratList = [RATIOk5, forgetfulRATIO, cTFT,cWSLS,GTFT,TF2T,T2FT,softMAJ,PROBER, RATIOTFT, ALT, cT20FT, squared_prop, exp_prop, PAD10, ForgivingCTFT, ForgivingSoftMAJ]*4 + [extort, extortD, ALLD]*10


Rounds=200
benefit=2
cost=1

PayoffTot=[0]*len(StratList)
WinTot=[0]*len(StratList)

for i in range(len(StratList)):
    for j in range(i+1,len(StratList)):
        Result=Game(StratList[i],StratList[j],Rounds)
        PayoffTot[i]=PayoffTot[i]+Result[0]
        PayoffTot[j]=PayoffTot[j]+Result[1]
        WinTot[i]=WinTot[i]+Result[2]
        WinTot[j]=WinTot[j]+Result[3]
        #print(StratList[i].__name__,"vs",StratList[j].__name__,":",StratList[i].__name__,"payoff=",Result[0],",",StratList[j].__name__,"payoff=",Result[1])
        
ScInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: PayoffTot[k])
print("Tournament Outcome by Score: \n")
for index, i in enumerate(ScInd):
        print(index+1,StratList[i].__name__,":","Score=",round(PayoffTot[i],3))
WinInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: WinTot[k])
print("\n \n Tournament Outcome by Wins: \n")
for index, i in enumerate(WinInd):
        print(index+1,StratList[i].__name__,":","Wins=",WinTot[i])

Tournament Outcome by Score: 

1 exp_prop : Score= 61.985
2 cTFT : Score= 61.9
3 exp_prop : Score= 61.83
4 exp_prop : Score= 61.6
5 cTFT : Score= 61.335
6 cTFT : Score= 61.21
7 cTFT : Score= 61.1
8 squared_prop : Score= 60.905
9 squared_prop : Score= 60.705
10 exp_prop : Score= 60.685
11 squared_prop : Score= 60.28
12 RATIOk5 : Score= 60.19
13 RATIOTFT : Score= 59.93
14 forgetfulRATIO : Score= 59.81
15 T2FT : Score= 59.8
16 T2FT : Score= 59.705
17 exp_prop_sqrt : Score= 59.655
18 exp_prop_sqrt : Score= 59.445
19 T2FT : Score= 59.425
20 forgetfulRATIO : Score= 59.41
21 T2FT : Score= 59.38
22 exp_prop_sqrt : Score= 59.355
23 squared_prop : Score= 59.325
24 forgetfulRATIO : Score= 59.155
25 softMAJ : Score= 59.065
26 exp_prop_sqrt : Score= 59.065
27 RATIOTFT : Score= 58.99
28 softMAJ : Score= 58.84
29 RATIOTFT : Score= 58.385
30 RATIOk5 : Score= 58.3
31 PAD10 : Score= 58.24
32 RATIOk5 : Score= 58.075
33 forgetfulRATIO : Score= 57.87
34 softMAJ : Score= 57.83
35 RATIOTFT : Score= 57.57
36 

In [19]:
13/14

0.9285714285714286

In [22]:
25/27

0.9259259259259259

In [66]:
StratList = [RATIO, TieredResponse, cTFT,dTFT,cWSLS,dWSLS,GTFT,TF2T,T2FT,GRIM,GRADUAL,softMAJ,hardMAJ,PROBER, ALT, RATIO2, cT20FT, PAD1, Update_Strat, squared_prop, exp_prop, PAD10, forgetfulRATIO, RATIOk5, ForgivingCTFT, ForgivingSoftMAJ]*3 + [extort, extortD, ALLD]*20
StratList += TeamOrganise(*[ScoreRunnerFinal, ScorePaceMakerFinal])
# StratList += TeamOrganise(*ScoreTeam9())
# StratList += TeamOrganise(*ScoreTeam8())
# StratList += TeamOrganise(*ScoreTeam7())
# StratList += TeamOrganise(*ScoreTeam6())

Rounds=200
benefit=2
cost=1

PayoffTot=[0]*len(StratList)
WinTot=[0]*len(StratList)

for i in range(len(StratList)):
    for j in range(i+1,len(StratList)):
        Result=Game(StratList[i],StratList[j],Rounds)
        PayoffTot[i]=PayoffTot[i]+Result[0]
        PayoffTot[j]=PayoffTot[j]+Result[1]
        WinTot[i]=WinTot[i]+Result[2]
        WinTot[j]=WinTot[j]+Result[3]
        #print(StratList[i].__name__,"vs",StratList[j].__name__,":",StratList[i].__name__,"payoff=",Result[0],",",StratList[j].__name__,"payoff=",Result[1])
        
ScInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: PayoffTot[k])
print("Tournament Outcome by Score: \n")
for index, i in enumerate(ScInd):
        print(index+1,StratList[i].__name__,":","Score=",round(PayoffTot[i],3))
WinInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: WinTot[k])
print("\n \n Tournament Outcome by Wins: \n")
for index, i in enumerate(WinInd):
        print(index+1,StratList[i].__name__,":","Wins=",WinTot[i])

Tournament Outcome by Score: 

1 ScoreRunnerFinal : Score= 92.65
2 ScoreRunnerFinal : Score= 91.78
3 ForgivingSoftMAJ : Score= 79.165
4 ForgivingSoftMAJ : Score= 78.775
5 RATIOk5 : Score= 78.675
6 RATIOk5 : Score= 77.96
7 forgetfulRATIO : Score= 77.515
8 RATIO : Score= 77.33
9 forgetfulRATIO : Score= 77.195
10 forgetfulRATIO : Score= 76.75
11 RATIO : Score= 76.44
12 RATIO : Score= 76.09
13 softMAJ : Score= 75.955
14 ForgivingSoftMAJ : Score= 75.715
15 softMAJ : Score= 75.65
16 RATIOk5 : Score= 75.45
17 softMAJ : Score= 74.625
18 ForgivingCTFT : Score= 74.51
19 squared_prop : Score= 74.405
20 ForgivingCTFT : Score= 74.05
21 T2FT : Score= 73.54
22 ForgivingCTFT : Score= 73.33
23 squared_prop : Score= 72.945
24 exp_prop : Score= 72.79
25 T2FT : Score= 72.74
26 PAD1 : Score= 72.265
27 T2FT : Score= 72.24
28 cTFT : Score= 71.295
29 exp_prop : Score= 71.125
30 cTFT : Score= 71.005
31 squared_prop : Score= 70.93
32 cTFT : Score= 70.105
33 exp_prop : Score= 69.7
34 RATIO2 : Score= 69.185
35 PA

In [48]:
StratList = [cTFT,dTFT,cWSLS,dWSLS,GTFT,TF2T,T2FT,GRIM,GRADUAL,softMAJ,hardMAJ,PROBER, ForgivingSoftMAJ, RATIOk5]*3 + [extort, ALLD]*30
# StratList += TeamOrganise(*ScoreTeam10())
# StratList += TeamOrganise(*ScoreTeam9())
# StratList += TeamOrganise(*ScoreTeam8())
# StratList += TeamOrganise(*ScoreTeam7())
# StratList += TeamOrganise(*ScoreTeam6())

Rounds=200
benefit=2
cost=1

PayoffTot=[0]*len(StratList)
WinTot=[0]*len(StratList)

for i in range(len(StratList)):
    for j in range(i+1,len(StratList)):
        Result=Game(StratList[i],StratList[j],Rounds)
        PayoffTot[i]=PayoffTot[i]+Result[0]
        PayoffTot[j]=PayoffTot[j]+Result[1]
        WinTot[i]=WinTot[i]+Result[2]
        WinTot[j]=WinTot[j]+Result[3]
        #print(StratList[i].__name__,"vs",StratList[j].__name__,":",StratList[i].__name__,"payoff=",Result[0],",",StratList[j].__name__,"payoff=",Result[1])
        
ScInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: PayoffTot[k])
print("Tournament Outcome by Score: \n")
for index, i in enumerate(ScInd):
        print(index+1,StratList[i].__name__,":","Score=",round(PayoffTot[i],3))
WinInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: WinTot[k])
print("\n \n Tournament Outcome by Wins: \n")
for index, i in enumerate(WinInd):
        print(index+1,StratList[i].__name__,":","Wins=",WinTot[i])

Tournament Outcome by Score: 

1 RATIOk5 : Score= 53.685
2 softMAJ : Score= 53.305
3 RATIOk5 : Score= 52.8
4 ForgivingSoftMAJ : Score= 52.78
5 ForgivingSoftMAJ : Score= 52.74
6 ForgivingSoftMAJ : Score= 52.185
7 RATIOk5 : Score= 52.02
8 softMAJ : Score= 48.485
9 softMAJ : Score= 48.225
10 cTFT : Score= 37.79
11 cTFT : Score= 37.575
12 TF2T : Score= 37.435
13 cTFT : Score= 37.23
14 TF2T : Score= 36.52
15 T2FT : Score= 35.97
16 TF2T : Score= 35.885
17 T2FT : Score= 35.7
18 T2FT : Score= 35.68
19 GTFT : Score= 35.615
20 GTFT : Score= 35.535
21 hardMAJ : Score= 33.875
22 GRADUAL : Score= 33.62
23 hardMAJ : Score= 33.455
24 GRADUAL : Score= 33.355
25 GRADUAL : Score= 33.32
26 GTFT : Score= 33.31
27 GRIM : Score= 33.265
28 GRIM : Score= 33.04
29 GRIM : Score= 33.005
30 hardMAJ : Score= 29.615
31 PROBER : Score= 24.31
32 PROBER : Score= 24.085
33 PROBER : Score= 23.94
34 extort : Score= 22.465
35 cWSLS : Score= 22.45
36 extort : Score= 21.93
37 cWSLS : Score= 21.895
38 cWSLS : Score= 21.875
3

In [50]:
StratList = [RATIO, TieredResponse, cTFT,dTFT,cWSLS,dWSLS,GTFT,TF2T,T2FT,GRIM,GRADUAL,softMAJ,hardMAJ,PROBER, ALT, RATIO2, cT20FT, PAD1, Update_Strat, squared_prop, exp_prop]*3 + [extort, ALLD]*30
StratList += TeamOrganise(*ScoreTeam10())
StratList += TeamOrganise(*ScoreTeam9())
StratList += TeamOrganise(*ScoreTeam8())
StratList += TeamOrganise(*ScoreTeam7())
StratList += TeamOrganise(*ScoreTeam6())

Rounds=200
benefit=2
cost=1

PayoffTot=[0]*len(StratList)
WinTot=[0]*len(StratList)

for i in range(len(StratList)):
    for j in range(i+1,len(StratList)):
        Result=Game(StratList[i],StratList[j],Rounds)
        PayoffTot[i]=PayoffTot[i]+Result[0]
        PayoffTot[j]=PayoffTot[j]+Result[1]
        WinTot[i]=WinTot[i]+Result[2]
        WinTot[j]=WinTot[j]+Result[3]
        #print(StratList[i].__name__,"vs",StratList[j].__name__,":",StratList[i].__name__,"payoff=",Result[0],",",StratList[j].__name__,"payoff=",Result[1])
        
ScInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: PayoffTot[k])
print("Tournament Outcome by Score: \n")
for index, i in enumerate(ScInd):
        print(index+1,StratList[i].__name__,":","Score=",round(PayoffTot[i],3))
WinInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: WinTot[k])
print("\n \n Tournament Outcome by Wins: \n")
for index, i in enumerate(WinInd):
        print(index+1,StratList[i].__name__,":","Wins=",WinTot[i])

SuperSecretCodes:  [['c', 'c', 'c', 'c', 'd', 'c', 'd', 'd', 'c', 'd'], ['d', 'd', 'c', 'd', 'd', 'c', 'd', 'd', 'd', 'd']]
SuperSecretCodes:  [['d', 'd', 'd', 'c', 'c', 'c', 'd', 'd', 'd'], ['c', 'c', 'd', 'c', 'd', 'd', 'd', 'd', 'd']]
SuperSecretCodes:  [['c', 'c', 'd', 'c', 'd', 'c', 'd', 'c'], ['c', 'd', 'c', 'd', 'c', 'c', 'd', 'c']]
SuperSecretCodes:  [['c', 'd', 'd', 'd', 'c', 'c', 'c'], ['d', 'd', 'c', 'c', 'c', 'c', 'c']]
SuperSecretCodes:  [['c', 'd', 'd', 'c', 'c', 'd'], ['d', 'c', 'd', 'c', 'c', 'd']]
Tournament Outcome by Score: 

1 ScoreRunner8 : Score= 88.19
2 ScoreRunner7 : Score= 87.25
3 ScoreRunner7 : Score= 87.195
4 ScoreRunner10 : Score= 86.655
5 ScoreRunner8 : Score= 85.9
6 ScoreRunner10 : Score= 83.92
7 ScoreRunner6 : Score= 83.54
8 ScoreRunner6 : Score= 81.89
9 RATIO : Score= 74.69
10 RATIO : Score= 74.09
11 RATIO : Score= 72.38
12 softMAJ : Score= 72.2
13 softMAJ : Score= 71.56
14 softMAJ : Score= 70.83
15 squared_prop : Score= 68.085
16 squared_prop : Score= 6

In [51]:
StratList = [RATIO, TieredResponse, cTFT,dTFT,cWSLS,dWSLS,GTFT,TF2T,T2FT,GRIM,GRADUAL,softMAJ,hardMAJ,PROBER, ALT, RATIO2, cT20FT, PAD1, Update_Strat, squared_prop, exp_prop]*3 + [extort, ALLD]*30
StratList += TeamOrganise(*ScoreTeam10())
StratList += TeamOrganise(*ScoreTeam9())
StratList += TeamOrganise(*ScoreTeam8())
StratList += TeamOrganise(*ScoreTeam7())
StratList += TeamOrganise(*ScoreTeam6())

Rounds=200
benefit=2
cost=1

PayoffTot=[0]*len(StratList)
WinTot=[0]*len(StratList)

for i in range(len(StratList)):
    for j in range(i+1,len(StratList)):
        Result=Game(StratList[i],StratList[j],Rounds)
        PayoffTot[i]=PayoffTot[i]+Result[0]
        PayoffTot[j]=PayoffTot[j]+Result[1]
        WinTot[i]=WinTot[i]+Result[2]
        WinTot[j]=WinTot[j]+Result[3]
        #print(StratList[i].__name__,"vs",StratList[j].__name__,":",StratList[i].__name__,"payoff=",Result[0],",",StratList[j].__name__,"payoff=",Result[1])
        
ScInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: PayoffTot[k])
print("Tournament Outcome by Score: \n")
for index, i in enumerate(ScInd):
        print(index+1,StratList[i].__name__,":","Score=",round(PayoffTot[i],3))
WinInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: WinTot[k])
print("\n \n Tournament Outcome by Wins: \n")
for index, i in enumerate(WinInd):
        print(index+1,StratList[i].__name__,":","Wins=",WinTot[i])

SuperSecretCodes:  [['c', 'c', 'd', 'd', 'd', 'd', 'c', 'c', 'c', 'c'], ['c', 'c', 'd', 'd', 'd', 'd', 'c', 'c', 'd', 'c']]
SuperSecretCodes:  [['d', 'd', 'c', 'c', 'd', 'c', 'd', 'd', 'd'], ['c', 'c', 'c', 'd', 'c', 'd', 'd', 'c', 'c']]
SuperSecretCodes:  [['c', 'd', 'd', 'd', 'd', 'd', 'd', 'd'], ['d', 'd', 'c', 'c', 'c', 'c', 'd', 'd']]
SuperSecretCodes:  [['c', 'd', 'd', 'd', 'd', 'd', 'c'], ['c', 'c', 'd', 'c', 'd', 'c', 'd']]
SuperSecretCodes:  [['d', 'c', 'd', 'c', 'c', 'd'], ['c', 'd', 'c', 'd', 'c', 'd']]
Tournament Outcome by Score: 

1 ScoreRunner8 : Score= 88.085
2 ScoreRunner8 : Score= 86.955
3 ScoreRunner10 : Score= 77.04
4 ScoreRunner10 : Score= 74.3
5 RATIO : Score= 73.85
6 RATIO : Score= 72.93
7 softMAJ : Score= 71.435
8 ScoreRunner7 : Score= 71.31
9 RATIO : Score= 70.985
10 softMAJ : Score= 70.78
11 softMAJ : Score= 69.555
12 squared_prop : Score= 69.385
13 squared_prop : Score= 68.345
14 squared_prop : Score= 68.225
15 exp_prop : Score= 67.24
16 ScoreRunner7 : Score=